In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
# encoding: utf-8
import pymongo, json
import pandas as pd
from pandas import Series, DataFrame
from scipy.stats import pearsonr
import numpy as np

pd.set_option('display.width', None)  # 设置字符显示宽度
pd.set_option('display.max_rows', None)  # 设置显示最大行
pd.set_option('display.max_columns', None)  # 设置显示最大行

client = pymongo.MongoClient('localhost', 27017)
futures = client.futures3
position = futures.position
market = futures.market
unit = futures.unit

start = '20190101'
end = '20200206'
var = 'JD'

position = DataFrame(list(position.find({'date': {'$gte': start}},'var':var)))
market = DataFrame(list(market.find({'date': {'$gte': start}},'var':var)))
unit = DataFrame(list(unit.find()))
# position = position[position['long_party_name'].notna()]

var1 = position[position['variety'] == var]

long_party_name = var1['long_party_name']
short_party_name = var1['short_party_name']
party_name = long_party_name.append(short_party_name)

long = var1.groupby(['date', 'variety', 'long_party_name'])[['long_open_interest', 'long_open_interest_chg']].sum()
# print(long)
short = var1.groupby(['date', 'variety', 'short_party_name'])[['short_open_interest', 'short_open_interest_chg']].sum()
# 合并
frames = [long, short]
position1 = pd.concat(frames, axis=1, sort=True).fillna(0).reset_index()
# print(position.head())
# 净持仓
position1['当日净持仓'] = position1.apply(lambda x: x['long_open_interest'] - x['short_open_interest'], axis=1)
# 字段更名
position2 = position1.rename(columns={'level_0': 'date', 'level_1': 'variety', 'level_2': 'mem'})
var2 = position2[position2['variety'] == var]

df = pd.DataFrame()

for i in party_name:
    try:
        mem = var2[var2['mem'] == i]
        position_behind = mem.shift(-1)
        # print(position_behind)

# 合并滞后和原始数据
        all_position = pd.merge(var2, position_behind, right_index=True, left_index=True)
        all_position = all_position[['date_y', 'variety_y', 'mem_y', 'long_open_interest_y','long_open_interest_chg_y', 'short_open_interest_y', 'short_open_interest_chg_y','当日净持仓_y', '当日净持仓_x']].dropna()
#
#         # 更名
        all_position= all_position.rename(columns={'date_y': 'date', 'variety_y': 'variety', 'mem_y': '会员',
                                                    'long_open_interest_y': '多单量', 'long_open_interest_chg_y':'多单增减量','short_open_interest_y': '空单量','short_open_interest_chg_y':'空单增减量',
                                                    '当日净持仓_y': '当日净持仓', '当日净持仓_x': '昨日净持仓'})
        # print(all_position)
        all_position['净持仓变化量'] = all_position.apply(lambda x: x['当日净持仓'] - x['昨日净持仓'], axis=1)

        # print(all_position.head())
        market['change'] = market.apply(lambda x: x['close'] - x['open'], axis=1)

        var3 = market[market['variety'] == var]

        chg = var3[['date', 'variety', 'change']]
        chg = chg.groupby(['date', 'variety'])['change'].sum()

        hb = pd.merge(chg, all_position, on=['date', 'variety'], how='outer').dropna().drop_duplicates()

        chgs = hb['change']
        nets = hb['净持仓变化量']
        p = chgs.corr(nets)

        hb = hb[hb['date'] <= end]
#
        d = [{'var': var,'start': start,'end':end,'会员': i,'相关性': p,'样本':hb['净持仓变化量'].count(),'多单量':hb['多单量'].values[-1],'多单增减量':hb['多单增减量'].values[-1],'空单量':hb['空单量'].values[-1],'空单增减量':hb['空单增减量'].values[-1],'当日净持仓': hb['当日净持仓'].values[-1], '昨日净持仓': hb['昨日净持仓'].values[-1], '净持仓变化量': hb['净持仓变化量'].values[-1]}]
        df1=pd.DataFrame(d)
        df=df.append(df1)

    except:
        continue
        
df.head()

In [ ]:
df = df.sort_values('相关性', ascending=False)
mean=df.groupby(['var','start','end'])['样本'].mean()[0]
df=df[df['样本'] >= mean]
df

In [ ]:
df=df[df['相关性']>0]

df

In [ ]:
corr=df.groupby(['var','start','end'])['相关性'].mean()[0]
corr

In [ ]:
df=df[df['相关性'] >= corr]
df

In [ ]:
part_sum=df.groupby(['var','start','end'])['会员'].sum()
part_sum=part_sum.reset_index()
netss=df.groupby(['var','start','end'])[['多单量','多单增减量','空单量','空单增减量','当日净持仓','昨日净持仓','净持仓变化量']].sum()
netss['corr']=df['相关性'].mean()
data=pd.merge(part_sum,netss,on=['end'])
data

# 市值

In [ ]:
#合约相乘

market['cv']=market.apply(lambda x:x['close'] * x['open_interest'],axis=1)
market=market.groupby(['date','variety'])[['cv','open_interest']].sum()
market['close_c']=market['cv']/market['open_interest']
market=market.reset_index()
market=pd.merge(market,unit[['variety','unit']],on=['variety'],how='outer')
market

In [ ]:
market['市值']=market.apply(lambda x:x['close_c'] * x['unit'],axis=1)
market

In [ ]:
position=pd.merge(position,market[['date','variety','市值']],on=['date','variety'],how='outer')
position

In [ ]:
position['价值']=position.apply(lambda x:x['净持仓']*x['市值'],axis=1)
position